In [19]:
import sys
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd ./drive/MyDrive/Colab Notebooks/unilevel

/content/drive/MyDrive/Colab Notebooks/unilevel


In [5]:
predict_product_info = pd.read_csv('./train_info.csv')
predict_sales = pd.read_csv('./train_sales.csv')

predict_product_info.drop(['material_name','material_name_zh',
                           'category', 'bar_code'], axis = 1, inplace=True)

predict_product_info.drop_duplicates(inplace=True)
predict_sales.drop_duplicates(inplace=True)

# predict_product_info.fillna(-1, inplace=True)
# predict_sales.fillna(-1, inplace=True)

# 处理info的重复uuid的问题
ndf = pd.DataFrame(columns = predict_product_info.columns) #根据df的列名建一个空表ndf
uuids = set(predict_product_info['uuid'])
for u in uuids:
    one = predict_product_info.loc[predict_product_info['uuid'] == u] #获取所有uid等于u的行，之后只会保存一行
    #在这里写if然后只保留一行，然后concat到ndf上，实现只保留一行
    olst = list(one['ingredient']) #或者用set
    zero = one.iloc[[0]] #iloc[行号]是series iloc[[行号]]是dataframe
    #zero['name']=str(olst)
    if len(olst) > 1: #等于1的就不用改了
        zero['ingredient'] = str(olst).replace('"[','').replace(']"','') #or =''.join(olst)
    ndf = pd.concat([ndf,zero]) #把选出来的zero加到ndf里

In [6]:
df_comb = predict_sales.merge(ndf, on='uuid')

In [7]:
print(df_comb.shape)
df_comb.head()

(150, 7)


,uuid,channel,sales_period_,sales_value,brand,launch_date,ingredient
0,112777,EC,6,8.201381,POND'S,2020-08-06,"['肉豆蔻酸', '甘油', '水', '丙二醇', '氢氧化钾', '棕榈酸', '乙二醇..."
1,112777,RT,6,5.497663,POND'S,2020-08-06,"['肉豆蔻酸', '甘油', '水', '丙二醇', '氢氧化钾', '棕榈酸', '乙二醇..."
2,112777,EC,12,8.525868,POND'S,2020-08-06,"['肉豆蔻酸', '甘油', '水', '丙二醇', '氢氧化钾', '棕榈酸', '乙二醇..."
3,112777,RT,12,5.531329,POND'S,2020-08-06,"['肉豆蔻酸', '甘油', '水', '丙二醇', '氢氧化钾', '棕榈酸', '乙二醇..."
4,157886,EC,6,8.775864,DOVE,2020-08-27,"['水', '甘油', '聚二甲基硅氧烷', '辛酸/癸酸甘油三酯', '棕榈酸', '乙二..."


In [32]:
final_id = df_comb[['uuid','channel','sales_period_']]
final_id

,uuid,channel,sales_period_
0,112777,EC,6
1,112777,RT,6
2,112777,EC,12
3,112777,RT,12
4,157886,EC,6
...,...,...,...
145,982584,RT,6
146,982584,EC,12
147,982584,RT,12
148,994949,RT,6


# 提取特征

In [9]:
## 成分
def ingredient(df_in):
    df = df_in.copy()
    s = set()
    for i in tqdm(df.ingredient):
        i = i.replace(" ",'').replace("',",' ').replace("'",'').replace("[",'').replace("]",'')
        i = set(i.split(' '))
        s = set.union(s, i)
    list_igd = list(s) # 共210种成分
    
    count_1 = -1
    for i in tqdm(df.ingredient):
        i = i.replace(" ",'').replace("',",' ').replace("'",'').replace("[",'').replace("]",'')
        i = i.split(' ')
        count_1 += 1
        df['ingredient'][count_1] = i
    
    for colname in tqdm(list_igd):
        count_2 = 0
        df['df_'+colname] = -1
        for sub in df['ingredient']:
            if colname in sub:
                df['df_'+colname][count_2] = 1
            else:
                df['df_'+colname][count_2] = 0
            count_2 += 1
    
    df_out = df.drop(['ingredient'],axis=1)
    
    return df_out

## 发行日期
def launch_date(df_in):
    df = df_in.copy()
    # str to date
    df['launch_date'] = pd.to_datetime(df['launch_date'])
    # find base date
    base_date = df['launch_date'].min() - datetime.timedelta(1)
    # calculate the lenght of launch date
    df['launch_date'] = df['launch_date'] - base_date
    # date to int
    df['launch_date'] = df['launch_date'].map(lambda x: x.days)
    
    df_out = df.copy()
    
    return df_out

## 品牌
def brand(df_in):
    df = df_in.copy()
    df = pd.concat([df, pd.get_dummies(df['brand'], prefix='brand')], axis=1)
    df_out = df.drop(['brand'],axis=1)
    
    return df_out

## 渠道
def channel(df_in):
    df = df_in.copy()
    df = pd.concat([df, pd.get_dummies(df['channel'], prefix='channel')], axis=1)
    df_out = df.drop(['channel'],axis=1)
    return df_out

In [10]:
df_proc = df_comb.copy()
print('Start processing...')
print('ingredient')
df_proc = ingredient(df_proc)
print('launch_date')
df_proc = launch_date(df_proc)
print('brand')
df_proc = brand(df_proc)
print('channel')
df_proc = channel(df_proc)
print('processing finish!')

Start processing...
ingredient


100%|██████████| 210/210 [00:18<00:00, 11.40it/s]

launch_date
brand
channel
processing finish!


# normalization

In [12]:
df_nor = df_proc.copy()

Y = df_nor[['uuid','sales_value']]
X = df_nor.drop(['sales_value','uuid'],axis=1)

ss = StandardScaler()
std_data = ss.fit_transform(X)
origin_data = ss.inverse_transform(std_data)

df_std_ = pd.DataFrame(std_data)
df_std = pd.concat([Y, df_std_], axis=1)

df_std

,uuid,sales_value,0,1,2,3,4,5,6,7,...,208,209,210,211,212,213,214,215,216,217
0,112777,8.201381,-1.0,-0.721947,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,1.191367,-1.191367
1,112777,5.497663,-1.0,-0.721947,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,-0.839372,0.839372
2,112777,8.525868,1.0,-0.721947,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,1.191367,-1.191367
3,112777,5.531329,1.0,-0.721947,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,-0.839372,0.839372
4,157886,8.775864,-1.0,-0.562292,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,-0.457905,0.5,-0.204124,2.412091,1.259447,-0.204124,-0.520756,-0.750000,1.191367,-1.191367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,982584,9.505179,-1.0,-1.854741,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,-0.839372,0.839372
146,982584,9.870377,1.0,-1.854741,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,1.191367,-1.191367
147,982584,9.844962,1.0,-1.854741,-0.910642,-0.267261,-0.320844,-0.267261,-0.793999,-0.165521,...,2.183857,0.5,-0.204124,-0.414578,-0.793999,-0.204124,1.920286,-0.750000,-0.839372,0.839372
148,994949,9.887114,-1.0,1.300357,1.098127,-0.267261,-0.320844,-0.267261,1.259447,-0.165521,...,-0.457905,0.5,-0.204124,-0.414578,-0.793999,-0.204124,-0.520756,1.333333,-0.839372,0.839372


# PCA降维

In [20]:
Y = df_std[['uuid','sales_value']]
X = df_std.drop(['uuid','sales_value'],axis=1)

pca = PCA(n_components=0.9)
reduced_X = pca.fit_transform(X)

Xp90 = pd.DataFrame(reduced_X)
con90 = pd.concat([Y, Xp90], axis=1)
con90

,uuid,sales_value,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,112777,8.201381,-5.410207,4.195662,3.929132,-1.858791,4.855210,-5.398792,0.101126,2.413454,9.630349,5.234299,0.809882,0.209280,0.985372,8.986844,-3.474554,-0.077471
1,112777,5.497663,-5.408785,4.129841,3.937356,-1.848919,4.950284,-5.425074,0.130033,2.346418,9.478475,5.282524,0.878587,0.141430,0.874914,8.941591,-3.573180,-0.215901
2,112777,8.525868,-5.410207,4.195662,3.929132,-1.858791,4.855210,-5.398792,0.101126,2.413454,9.630349,5.234299,0.809882,0.209280,0.985372,8.986844,-3.474554,-0.077471
3,112777,5.531329,-5.408785,4.129841,3.937356,-1.848919,4.950284,-5.425074,0.130033,2.346418,9.478475,5.282524,0.878587,0.141430,0.874914,8.941591,-3.573180,-0.215901
4,157886,8.775864,1.842251,3.397010,-0.319287,-1.324899,-0.741778,2.259070,-2.344235,1.943970,1.108271,-0.782064,-3.324806,-1.408970,-3.499148,0.754680,1.900294,-0.387174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,982584,9.505179,-4.831671,3.148019,2.828306,-0.757041,3.742662,-4.289876,-0.301363,1.885886,4.183063,1.997834,0.641796,0.232990,-0.714070,-6.275873,2.330244,-0.252371
146,982584,9.870377,-4.833093,3.213840,2.820082,-0.766913,3.647588,-4.263594,-0.330270,1.952921,4.334937,1.949610,0.573091,0.300840,-0.603612,-6.230620,2.428870,-0.113941
147,982584,9.844962,-4.831671,3.148019,2.828306,-0.757041,3.742662,-4.289876,-0.301363,1.885886,4.183063,1.997834,0.641796,0.232990,-0.714070,-6.275873,2.330244,-0.252371
148,994949,9.887114,0.961780,-8.580597,-5.761107,1.867975,2.758202,-1.854569,0.666395,-0.797810,-0.736314,-0.587281,0.318738,-0.090328,-0.712399,0.393450,-1.935920,-1.608698


# 评估模型

In [16]:
df_model = con90.copy()

x = df_model.drop(['uuid','sales_value'],axis=1)
y = df_model[['uuid','sales_value']]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=2022)
y_train_id = y_train.uuid
y_test_id = y_test.uuid

y_train.drop(['uuid'], axis=1, inplace=True)
y_test.drop(['uuid'], axis=1, inplace=True)

In [17]:
lr = LinearRegression()
xgb = XGBRegressor()
models =[lr, xgb]

for model in models:
    model=model.fit(x_train,y_train)
    #predict_train = model.predict(x_train)
    predict_test = model.predict(x_test)
    print(model)
    print('Accureacy:', np.sqrt(metrics.mean_squared_error(y_test,predict_test))) # RMSE
    print('**********************************')

LinearRegression()
Accureacy: 1.441241078378971
**********************************
[16:38:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor()
Accureacy: 0.6980910599213737
**********************************


# 训练全量数据

In [21]:
df_model = con90.copy()

x = df_model.drop(['uuid','sales_value'],axis=1)
y = df_model['sales_value']

xgb = XGBRegressor()
model = xgb.fit(x,y)

# 保存模型
joblib.dump(model, './xgboost.pkl')

[16:40:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


['./xgboost.pkl']

# 读取模型

In [22]:
xgb = joblib.load('./xgboost.pkl')

[16:41:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [23]:
predict_result = xgb.predict(x)
print(xgb)
print(predict_result)

XGBRegressor(missing=nan)
[ 8.135774   5.8039727  8.135774   5.8039727  8.904807   8.567183
  8.904807   8.567183   9.79875    9.643942   9.79875    9.643942
 10.527485  10.105641  10.527485  10.105641   8.143913   5.861423
  8.143913   5.861423   8.52131    8.52131    9.97329    9.97329
 10.013108   9.048699  10.013108   9.048699   7.6789713  6.9584303
  7.6789713  6.9584303 10.691438  10.602553  10.691438  10.602553
  9.833087   9.477027   9.833087   9.477027  10.082016  10.03555
 10.082016  10.03555    9.390448   9.077755   9.390448   9.077755
  7.856399   7.040256   7.856399   7.040256   9.652326   9.585949
  9.652326   9.585949   9.237164   9.237164   9.092919   7.5146675
  9.092919   7.5146675 11.749282  11.190799  11.749282  11.190799
  9.119799   7.534433   9.119799   7.534433   9.106087   9.418269
  9.106087   9.418269   9.375919   9.233635   9.375919   9.233635
 10.617355   7.973742  10.617355   7.973742  11.200318  11.200318
  9.10724    9.72507    9.10724    9.72507    9.86

# 生成最终结果

In [33]:
final_id.insert(3,'predict_result',predict_result)
df_final = pd.merge(predict_sales,final_id,
                    on=['uuid','channel','sales_period_'],
                    how = 'outer')

df_final['predict_result'].fillna(3.96, inplace = True)

print(df_final.info())
df_final

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 161
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   uuid            162 non-null    object 
 1   channel         162 non-null    object 
 2   sales_period_   162 non-null    int64  
 3   sales_value     162 non-null    float64
 4   predict_result  162 non-null    float32
dtypes: float32(1), float64(1), int64(1), object(2)
memory usage: 7.0+ KB
None


,uuid,channel,sales_period_,sales_value,predict_result
0,112777,EC,6,8.201381,8.135774
1,112777,RT,6,5.497663,5.803973
2,157886,EC,6,8.775864,8.904807
3,157886,RT,6,7.876226,8.567183
4,162194,EC,6,10.007262,9.798750
...,...,...,...,...,...
157,962209,EC,12,10.235670,9.897096
158,962209,RT,12,10.067857,9.864491
159,982584,EC,12,9.870377,9.839612
160,982584,RT,12,9.844962,9.827997
